In [1]:
%%capture
from source.active_learning.active_learning_mnist_batch import *

In [2]:
def cnn():
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(10, activation="softmax")
    ])

    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

In [3]:

def dropout_weights(weights, ratio):
    dropout_mask = [np.random.binomial(1, 1 - ratio, w.shape) for w in weights]
    return [w * mask for w, mask in zip(weights, dropout_mask)]


def eval_prioritization_strategy(data, model, prioritizer, verbose=True):
    """

    :param model:
    :param prioritizer: lamdba: indices, predictions: sorted indices, ie. query strategy
    :return:
    """
    query_steps = 20
    query_size = 16

    x_train, y_train, x_test, y_test = data

    train_indices = range(60000)

    # First subset of data
    selected_indices = train_indices[0:query_size]
    x_train_subset = x_train[selected_indices, ...]
    y_train_subset = y_train[selected_indices, ...]

    test_accuracies = []
    for i in range(query_steps):
        selected_indices = train_indices[0:query_size]

        if i > 0:
            x_train_subset = np.concatenate((x_train_subset, x_train[selected_indices, ...]))
            y_train_subset = np.concatenate((y_train_subset, y_train[selected_indices, ...]))

        model.fit(x_train_subset, y_train_subset, epochs=5, verbose=0)

        loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
        test_accuracies.append(accuracy)

        if verbose:
            print('Training data size of %d => accuracy %f' % (x_train_subset.shape[0], accuracy))

        # prepare for the next step of the loop
        rest_indices = train_indices[query_size:]

        # copy the models 5 times, all weights of the model is dropout with a ratio of 0.2:
        predictions = []
        for i in range(5):
            mc_model = tf.keras.models.clone_model(model)
            dropped_out_weights = dropout_weights(mc_model.get_weights(), 0.2)
            mc_model.set_weights(dropped_out_weights)
            predictions.append(mc_model.predict(x_train[rest_indices, ...]))

        predictions = np.transpose(predictions, (1, 0, 2))  # return (N_samples, K_models, C_classes)
        train_indices = prioritizer(rest_indices, predictions, query_size)

    print("- finished -")
    return test_accuracies

In [4]:
# prepare data
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0
data = x_train, y_train, x_test, y_test

In [ ]:
# get results of different AL method
acc_baseline = eval_prioritization_strategy(data, mlp_classifier(), trivial_strategy)
acc_bald = eval_prioritization_strategy(data, mlp_classifier(), bald_strategy)
acc_batchbald = eval_prioritization_strategy(data, mlp_classifier(), batchbald_strategy)


Training data size of 16 => accuracy 0.306500
Training data size of 32 => accuracy 0.430200
Training data size of 48 => accuracy 0.567700
Training data size of 64 => accuracy 0.612400
Training data size of 80 => accuracy 0.647700
Training data size of 96 => accuracy 0.663800
Training data size of 112 => accuracy 0.700600
Training data size of 128 => accuracy 0.712200
Training data size of 144 => accuracy 0.729200
Training data size of 160 => accuracy 0.737500
Training data size of 176 => accuracy 0.746200


In [ ]:

# visualize the performance difference
plt.plot(acc_baseline, 'k', label='baseline')
plt.plot(acc_bald, 'b', label='bald')
plt.plot(acc_batchbald, 'g', label='batchbald')
plt.legend()
